In [85]:
import itertools
import joblib
import pickle 

import networkx as nx
import numpy as np
import pandas as pd
import cvxpy as cp

import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import seaborn as sns
import random 
import joblib

import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import mean_squared_error

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import evaluation_utils as evut
import opt_utils as oput
import params

np.random.seed(0)

In [2]:
M_base = joblib.load('batteries/scms/M_WMG_bins_5_avg_2.pkl')
M_abst = joblib.load('batteries/scms/M_LRCS_bins_5.pkl') 

In [3]:
# Base-level interventions (new style)
iota0 = None
iota1 = ops.Intervention({'CG': 75.})
iota2 = ops.Intervention({'CG': 110.})
iota3 = ops.Intervention({'CG': 180.})
iota4 = ops.Intervention({'CG': 200.})

# Abstract-level interventions (new style)
iota0_prime = None
iota1_prime = ops.Intervention({'CG': 75.})
iota2_prime = ops.Intervention({'CG': 100.})
iota3_prime = ops.Intervention({'CG': 200.})

# Mapping
omega = {
    iota0: iota0_prime,
    iota1: iota1_prime,
    iota2: iota2_prime,
    iota3: iota3_prime,
    iota4: iota3_prime
}

Ill = list(set(omega.keys()))
Ihl = list(set(omega.values()))


In [4]:
df_base = joblib.load('batteries/dfs/df_WMG_bins_5_avg_2.pkl')
df_abst = joblib.load('batteries/dfs/df_LRCS_bins_5.pkl')

In [5]:
df_base.drop(df_base.columns[[1,2]], axis=1, inplace=True)
df_base.replace({75:0, 110:1, 150:2, 170:3, 180:4, 200:5}, inplace=True)

df_abst.drop(df_abst.columns[[1]], axis=1, inplace=True)
df_abst.replace({75:0, 100:1, 200:2}, inplace=True)

In [6]:
# Rename columns to match graph
df_base = df_base.rename(columns={
    'binned ML_avg0': 'ML0',
    'binned ML_avg1': 'ML1'
})
# Rename columns to match graph
df_abst = df_abst.rename(columns={
    'Comma gap (µm)': 'CG', 'binned ML': 'ML'
})

In [7]:
ll_coeffs = {}

for child in M_base.nodes():
    parents = M_base.get_parents(child)
    if not parents:
        continue  # skip root nodes
    X = df_base[parents].values
    y = df_base[child].values
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    for i, parent in enumerate(parents):
        ll_coeffs[(parent, child)] = coef[i]

Gll = CBN(list(ll_coeffs.keys()))

hl_coeffs = {}

for child in M_abst.nodes():
    parents = M_abst.get_parents(child)
    if not parents:
        continue  # skip root nodes
    X = df_abst[parents].values
    y = df_abst[child].values
    coef = np.linalg.lstsq(X, y, rcond=None)[0]
    for i, parent in enumerate(parents):
        hl_coeffs[(parent, child)] = coef[i]

Ghl = CBN(list(hl_coeffs.keys()))


In [8]:
min_samples = min(df_base.shape[0], df_abst.shape[0])
df_base = df_base[:min_samples]
df_abst = df_abst[:min_samples]

df_base= df_base.to_numpy()
df_abst= df_abst.to_numpy()

In [9]:
num_llsamples = df_base.shape[0]
num_hlsamples = df_abst.shape[0]
l = len(Gll.nodes())
h = len(Ghl.nodes())

In [10]:
U_ll_hat, mu_U_ll_hat, Sigma_U_ll_hat = mut.lan_abduction(df_base, Gll, ll_coeffs)
U_hl_hat, mu_U_hl_hat, Sigma_U_hl_hat = mut.lan_abduction(df_abst, Ghl, hl_coeffs)

In [11]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [12]:
L_matrices = oput.compute_struc_matrices(LLmodels, Ill)
H_matrices = oput.compute_struc_matrices(HLmodels, Ihl)

In [13]:
def compute_empirical_radius(N, eta, c1=1.0, c2=1.0, alpha=2.0, m=3):
    """
    Compute epsilon_N(eta) for empirical Wasserstein case.

    Parameters:
    - N: int, number of samples
    - eta: float, confidence level (0 < eta < 1)
    - c1: float, constant from theorem (default 1.0, adjust if needed)
    - c2: float, constant from theorem (default 1.0, adjust if needed)
    - alpha: float, light-tail exponent (P[exp(||ξ||^α)] ≤ A)
    - m: int, ambient dimension

    Returns:
    - epsilon: float, the concentration radius
    """
    assert 0 < eta < 1, "eta must be in (0,1)"
    threshold = np.log(c1 / eta) / c2
    if N >= threshold:
        exponent = min(1/m, 0.5)
    else:
        exponent = 1 / alpha

    epsilon = (np.log(c1 / eta) / (c2 * N)) ** exponent
    return epsilon

In [14]:
ll_bound = round(compute_empirical_radius(N=num_llsamples, eta=0.05, c1=1000.0, c2=1.0, alpha=2.0, m=l), 3)
hl_bound = round(compute_empirical_radius(N=num_hlsamples, eta=0.05, c1=1000.0, c2=1.0, alpha=2.0, m=h), 3)


In [15]:
epsilon, delta = ll_bound, hl_bound

eta_max = 0.001
eta_min = 0.001

max_iter = 1000
num_steps_min = 5
num_steps_max = 5

robust_L = True 
robust_H = True

initialization = 'random'

tol  = 1e-4
seed = 23

In [16]:
opt_params_erica = {
                        'U_L': U_ll_hat,
                        'U_H': U_hl_hat,
                        'L_models': LLmodels,
                        'H_models': HLmodels,
                        'omega': omega,
                        'epsilon': epsilon,
                        'delta': delta,
                        'eta_min': eta_min,
                        'eta_max': eta_max,
                        'num_steps_min': num_steps_min,
                        'num_steps_max': num_steps_max,
                        'max_iter': max_iter,
                        'tol': tol,
                        'seed': seed,
                        'robust_L': robust_L,
                        'robust_H': robust_H,
                        'initialization': initialization,
                        'experiment': 'battery_discrete'
                    }

In [17]:
# Define different epsilon=delta values
eps_delta_values     = [8, ll_bound, 1, 2, 4]
diroca_train_results_empirical = {}

# For each epsilon=delta value
for eps_delta in eps_delta_values:
    print(f"Training for ε=δ = {eps_delta}")
    # Update theta parameters
    if eps_delta == ll_bound:
        opt_params_erica['epsilon'] = ll_bound
        opt_params_erica['delta']   = hl_bound
    
    else:
        opt_params_erica['epsilon'] = eps_delta
        opt_params_erica['delta']   = eps_delta
    
    # Run ERICA optimization
    params_empirical, T_empirical = oput.run_empirical_erica_optimization(**opt_params_erica)
    
    # Store results including optimization parameters and transformation matrix
    if eps_delta == ll_bound:
        diroca_train_results_empirical['T_'+str(ll_bound)+'-'+str(hl_bound)] = {
                                                    'optimization_params': params_empirical,
                                                    'T_matrix': T_empirical
                                                }
    else:
        diroca_train_results_empirical['T_'+str(eps_delta)] = {
                                                    'optimization_params': params_empirical,
                                                    'T_matrix': T_empirical
                                                }

print("\nTraining completed. T matrices stored in trained_results dictionary.")
print("Available ε=δ values:", list(diroca_train_results_empirical.keys()))



Training for ε=δ = 8


 16%|█▌        | 158/1000 [00:03<00:19, 43.08it/s]


Converged at iteration 159
Training for ε=δ = 0.537


 27%|██▋       | 272/1000 [00:06<00:17, 42.61it/s]


Converged at iteration 273
Training for ε=δ = 1


...

Converged at iteration 267
Training for ε=δ = 2


 16%|█▌        | 158/1000 [00:03<00:17, 46.99it/s]


Converged at iteration 159
Training for ε=δ = 4


 16%|█▌        | 158/1000 [00:03<00:20, 40.15it/s]

Converged at iteration 159

Training completed. T matrices stored in trained_results dictionary.
Available ε=δ values: ['T_8', 'T_0.537-0.393', 'T_1', 'T_2', 'T_4']


In [18]:
params_enrico, T_enrico = oput.run_empirical_erica_optimization(**{**opt_params_erica, 'robust_L': False, 'robust_H': False})

100%|██████████| 1000/1000 [00:01<00:00, 589.21it/s]


In [19]:
diroca_train_results_empirical['T_0.00'] = {
                                'optimization_params': params_enrico,
                                'T_matrix': T_enrico
                            }

In [20]:
opt_params_bary = {
                        'U_ll_hat':U_ll_hat,
                        'U_hl_hat':U_hl_hat,
                        'L_matrices':L_matrices,
                        'H_matrices':H_matrices,
                        'max_iter':max_iter,
                        'tol':tol,
                        'seed':seed
                    }
                                 

In [21]:
T_bary = oput.run_empirical_bary_optim(**opt_params_bary)
params_bary = {'L':{}, 'H':{}}

100%|██████████| 1000/1000 [00:00<00:00, 2549.54it/s]


In [22]:
diroca_train_results_empirical['T_b'] = {
                                'optimization_params': params_bary,
                                'T_matrix': T_bary
                            }

In [23]:
opt_params_smooth = {
                        'U_L': U_ll_hat,
                        'U_H': U_hl_hat,
                        'L_models': LLmodels,
                        'H_models': HLmodels,
                        'omega': omega,
                        'eta_min': eta_min,
                        'num_steps_min': num_steps_min,
                        'max_iter': max_iter,
                        'tol': tol,
                        'seed': seed,
                        'noise_sigma': 0.1,
                        'num_noise_samples': 10
                        }

In [24]:
params_smooth, T_smooth = oput.run_empirical_smooth_optimization(**opt_params_smooth)

100%|██████████| 1000/1000 [00:56<00:00, 17.84it/s]


In [25]:
diroca_train_results_empirical['T_s'] = {
                                'optimization_params': params_smooth,
                                'T_matrix': T_smooth
                            }

In [29]:
joblib.dump(diroca_train_results_empirical, f"data/'battery_discrete'/diroca_train_results_empirical.pkl")

["data/'battery_discrete'/diroca_train_results_empirical.pkl"]

In [82]:
def downstream_evaluation(T, df_base, df_abst):
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.metrics import mean_squared_error
    import numpy as np

    tau_samples  = T @ df_base.T
    abst_samples = df_abst.T

    # Step 1: Transpose to get (N, dim)
    X_real = abst_samples.T
    X_gen  = tau_samples.T

    # Step 2: Define target labels and intervention groupings
    y = df_abst[:, 1]
    groups = df_abst[:, 0]

    assert X_real.shape[0] == len(y) == len(groups), "Mismatch in number of samples"

    # Step 3: Combine real and generated data
    X_all = np.concatenate([X_real, X_gen], axis=0)
    y_all = np.concatenate([y, y], axis=0)
    groups_all  = np.concatenate([groups, groups], axis=0)

    logo = LeaveOneGroupOut()

    # Mode 1: Real → Real
    mse_real = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_real[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_real.append(mean_squared_error(y[test_idx], y_pred))

    # Mode 2: Augmented → Real
    mse_aug = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_gen[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_aug.append(mean_squared_error(y[test_idx], y_pred))

    # Mode 3: Real + Augmented → Real
    mse_mix = []
    for test_group in np.unique(groups):
        test_mask = (groups == test_group)
        test_idx_real = np.where(test_mask)[0]

        train_mask_real = (groups != test_group)
        train_idx_real = np.where(train_mask_real)[0]
        train_idx_gen = np.arange(len(y)) + len(y)
        train_idx_all = np.concatenate([train_idx_real, train_idx_gen])

        model = Lasso().fit(X_all[train_idx_all], y_all[train_idx_all])
        y_pred = model.predict(X_real[test_idx_real])
        mse_mix.append(mean_squared_error(y[test_idx_real], y_pred))

    return {
        "Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug": (np.mean(mse_aug), np.std(mse_aug)),
        "AugReal": (np.mean(mse_mix), np.std(mse_mix))
    }


In [84]:
for method in list(diroca_train_results_empirical.keys()):
    print(f'Method: {method}')
    T = diroca_train_results_empirical[method]['T_matrix']
    d = downstream_evaluation(T, df_base, df_abst)
    print(d)
    print('-------------------------------- \n')

Method: T_8
{'Real': (5.309079989742723, 2.7255673735214216), 'Aug': (5.55593643707483, 2.387732017528672), 'AugReal': (3.350116487661396, 2.577474204135139)}
-------------------------------- 

Method: T_0.537-0.393
{'Real': (5.309079989742723, 2.7255673735214216), 'Aug': (5.55593643707483, 2.387732017528672), 'AugReal': (3.5333412472364345, 2.425124596371633)}
-------------------------------- 

Method: T_1
{'Real': (5.309079989742723, 2.7255673735214216), 'Aug': (5.55593643707483, 2.387732017528672), 'AugReal': (3.5513524000111096, 2.4111322810427374)}
-------------------------------- 

Method: T_2
{'Real': (5.309079989742723, 2.7255673735214216), 'Aug': (5.55593643707483, 2.387732017528672), 'AugReal': (3.350116487661396, 2.577474204135139)}
-------------------------------- 

Method: T_4
{'Real': (5.309079989742723, 2.7255673735214216), 'Aug': (5.55593643707483, 2.387732017528672), 'AugReal': (3.350116487661396, 2.577474204135139)}
-------------------------------- 

Method: T_0.00
{'

In [106]:
def downstream_evaluation_with_noise(T, df_base, df_abst, noise_level=0.1, noise_in='base'):
    """
    Evaluate abstraction robustness with added noise.

    Args:
        T (np.ndarray): transformation matrix (h, d)
        df_base (np.ndarray): base-level data (N, d)
        df_abst (np.ndarray): abstract-level data (N, >=2) [0: group, 1: label, 2+: features]
        noise_level (float): standard deviation of Gaussian noise
        noise_in (str): where to add noise: 'base', 'both', or 'abst'

    Returns:
        dict: evaluation results (mean ± std for each mode)
    """
    
    # Copy inputs
    df_base_noisy = df_base.copy()
    df_abst_noisy = df_abst.copy()
    df_abst_noisy = df_abst_noisy.astype(float)


    if noise_in in ['base', 'both']:
        df_base_noisy += np.random.normal(0, noise_level, df_base.shape)

    if noise_in in ['abst', 'both']:
        df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)

    # Generate transformed and real abstract samples
    tau_samples  = T @ df_base_noisy.T
    #abst_samples = df_abst_noisy[:, 2:].T
    abst_samples = df_abst_noisy.T


    X_real = abst_samples.T
    X_gen  = tau_samples.T
    
    y = df_abst_noisy[:, 1]
    groups = df_abst_noisy[:, 0]

    assert X_real.shape[0] == len(y) == len(groups), "Mismatch in sample counts"

    X_all = np.concatenate([X_real, X_gen], axis=0)
    y_all = np.concatenate([y, y], axis=0)
    groups_all = np.concatenate([groups, groups], axis=0)

    logo = LeaveOneGroupOut()

    # Mode 1: Real → Real
    mse_real = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_real[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_real.append(mean_squared_error(y[test_idx], y_pred))

    # Mode 2: Augmented → Real
    mse_aug = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_gen[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_aug.append(mean_squared_error(y[test_idx], y_pred))

    # Mode 3: Real + Augmented → Real
    mse_mix = []
    for test_group in np.unique(groups):
        test_mask = (groups == test_group)
        test_idx_real = np.where(test_mask)[0]

        train_mask_real = (groups != test_group)
        train_idx_real = np.where(train_mask_real)[0]
        train_idx_gen = np.arange(len(y)) + len(y)
        train_idx_all = np.concatenate([train_idx_real, train_idx_gen])

        model = Lasso().fit(X_all[train_idx_all], y_all[train_idx_all])
        y_pred = model.predict(X_real[test_idx_real])
        mse_mix.append(mean_squared_error(y[test_idx_real], y_pred))

    return {
        "Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug": (np.mean(mse_aug), np.std(mse_aug)),
        "AugReal": (np.mean(mse_mix), np.std(mse_mix))
    }


In [112]:
for method in list(diroca_train_results_empirical.keys()):
    print(f'Method: {method}')
    T = diroca_train_results_empirical[method]['T_matrix']
    d = downstream_evaluation_with_noise(T, df_base, df_abst, noise_level=0.2, noise_in='both')

    print(d['AugReal'])
    print('-------------------------------- \n')

Method: T_8
(1.3246280648705357, 1.9357229836589438)
-------------------------------- 

Method: T_0.537-0.393
(1.4662915245116084, 2.235598606696959)
-------------------------------- 

Method: T_1
(1.3204628707649662, 2.0077542145624254)
-------------------------------- 

Method: T_2
(1.3596850290595017, 2.001499776578738)
-------------------------------- 

Method: T_4
(1.36985055408596, 1.999388245161991)
-------------------------------- 

Method: T_0.00
(1.4314357777070637, 2.257912829013462)
-------------------------------- 

Method: T_b
(1.3947828570843153, 2.007051436628062)
-------------------------------- 

Method: T_s
(1.3426993754464802, 2.184303951144872)
-------------------------------- 



In [114]:
def sample_from_frobenius_ball(params, radius_type, boundary, rand_rad):
    """
    Initialize a matrix inside the Frobenius ball with ||X||_F^2 <= N*epsilon^2
    """
    num_samples, num_vars = params['pert_U'].shape
    if radius_type == 'worst_case':
        radius = params['radius_worst']

    elif radius_type == 'hat_case':
        radius = params['radius']

    elif radius_type == 'random':
        radius = rand_rad
        
    matrix           = np.random.randn(num_samples, num_vars)  
    squared_norm     = np.linalg.norm(matrix, 'fro')**2
    max_squared_norm = num_samples * radius**2
    
    if boundary == True:
        scaling_factor = np.sqrt(max_squared_norm / squared_norm) * np.random.rand(1)
    else:
        scaling_factor = np.sqrt(max_squared_norm / squared_norm)
    return matrix * scaling_factor

In [115]:
def generate_pertubation(pert_type, pert_level, experiment, rad=None):
    
    params = load_empirical_optimization_params(experiment, pert_level)
    
    if pert_type == 'sample_radius':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=False, rand_rad=rad)
    
    elif pert_type == 'sample_radius_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=False, rand_rad=rad)

    elif pert_type == 'boundary_worst':
        P = params['pert_U']
        
    elif pert_type == 'boundary_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=True, rand_rad=rad)

    elif pert_type == 'random_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=False, rand_rad=rad)

    elif pert_type == 'random_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=False, rand_rad=rad)

    elif pert_type == 'boundary_random_hat':
        P = sample_from_frobenius_ball(params, 'hat_case', boundary=True, rand_rad=rad)

    elif pert_type == 'boundary_random_worst':
        P = sample_from_frobenius_ball(params, 'worst_case', boundary=True, rand_rad=rad)

    elif pert_type == 'random_normal':
        P = np.random.randn(*params['pert_U'].shape)

    return P

In [118]:
def downstream_evaluation_with_structured_noise(
    T, df_base, df_abst,
    noise_type='gaussian',  # or 'frobenius'
    noise_level=0.1,
    noise_in='base',  # 'base', 'abst', or 'both'
    pert_type=None,
    experiment=None,
    radius_L=None,
    radius_H=None
):
    """
    Evaluate abstraction robustness with either Gaussian or Frobenius-ball structured noise.
    """

    from sklearn.linear_model import Lasso
    from sklearn.model_selection import LeaveOneGroupOut
    from sklearn.metrics import mean_squared_error
    import numpy as np

    df_base_noisy = df_base.copy().astype(float)
    df_abst_noisy = df_abst.copy().astype(float)

    if noise_type == 'gaussian':
        if noise_in in ['base', 'both']:
            df_base_noisy += np.random.normal(0, noise_level, df_base.shape)
        if noise_in in ['abst', 'both']:
            df_abst_noisy += np.random.normal(0, noise_level, df_abst.shape)

    elif noise_type == 'frobenius':

        if noise_in in ['base', 'both']:
            assert pert_type is not None and experiment is not None
            pert_L = generate_pertubation(pert_type, 'L', experiment, rad=radius_L)
            df_base_noisy += pert_L
        if noise_in in ['abst', 'both']:
            assert pert_type is not None and experiment is not None
            pert_H = generate_pertubation(pert_type, 'H', experiment, rad=radius_H)
            df_abst_noisy += pert_H

    tau_samples = T @ df_base_noisy.T
    abst_samples = df_abst_noisy.T

    X_real = abst_samples.T
    X_gen = tau_samples.T

    y = df_abst_noisy[:, 1]
    groups = df_abst_noisy[:, 0]

    assert X_real.shape[0] == len(y) == len(groups), "Mismatch in sample counts"

    X_all = np.concatenate([X_real, X_gen], axis=0)
    y_all = np.concatenate([y, y], axis=0)
    groups_all = np.concatenate([groups, groups], axis=0)

    logo = LeaveOneGroupOut()

    mse_real = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_real[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_real.append(mean_squared_error(y[test_idx], y_pred))

    mse_aug = []
    for train_idx, test_idx in logo.split(X_real, y, groups=groups):
        model = Lasso().fit(X_gen[train_idx], y[train_idx])
        y_pred = model.predict(X_real[test_idx])
        mse_aug.append(mean_squared_error(y[test_idx], y_pred))

    mse_mix = []
    for test_group in np.unique(groups):
        test_mask = (groups == test_group)
        test_idx_real = np.where(test_mask)[0]

        train_mask_real = (groups != test_group)
        train_idx_real = np.where(train_mask_real)[0]
        train_idx_gen = np.arange(len(y)) + len(y)
        train_idx_all = np.concatenate([train_idx_real, train_idx_gen])

        model = Lasso().fit(X_all[train_idx_all], y_all[train_idx_all])
        y_pred = model.predict(X_real[test_idx_real])
        mse_mix.append(mean_squared_error(y[test_idx_real], y_pred))

    return {
        "Real→Real": (np.mean(mse_real), np.std(mse_real)),
        "Aug→Real": (np.mean(mse_aug), np.std(mse_aug)),
        "Real+Aug→Real": (np.mean(mse_mix), np.std(mse_mix))
    }


In [119]:
# Gaussian noise in base-level only
downstream_evaluation_with_structured_noise(T, df_base, df_abst, noise_type='gaussian', noise_level=0.5, noise_in='base')

# Frobenius-ball structured noise (requires experiment + params)
downstream_evaluation_with_structured_noise(
    T, df_base, df_abst,
    noise_type='frobenius',
    noise_in='both',
    pert_type='random_worst',
    experiment='MyExperiment',
    radius_L=3,
    radius_H=2
)


NameError: name 'load_empirical_optimization_params' is not defined